<a href="https://colab.research.google.com/github/Triniti0/klasifikasi-penyakit-daun-cabai/blob/main/Eksperimen_Optimasi_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eksperimen Optimasi 1

Inisiasi libary dan konfigurasi awal

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
IMG_SIZE = 224
AUTOTUNE = tf.data.AUTOTUNE

def load_dataset(batch_size):

    train_ds = tf.keras.utils.image_dataset_from_directory(
        "/content/drive/MyDrive/Seminar Hasil/dataset_processed/train",
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=batch_size
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        "/content/drive/MyDrive/Seminar Hasil/dataset_processed/val",
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=batch_size
    )

    test_ds = tf.keras.utils.image_dataset_from_directory(
        "/content/drive/MyDrive/Seminar Hasil/dataset_processed/test",
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=batch_size,
        shuffle=False
    )

    class_names = train_ds.class_names

    def preprocess(x, y):
        return preprocess_input(x), y

    train_ds = train_ds.map(preprocess).prefetch(AUTOTUNE)
    val_ds = val_ds.map(preprocess).prefetch(AUTOTUNE)
    test_ds = test_ds.map(preprocess).prefetch(AUTOTUNE)

    return train_ds, val_ds, test_ds, class_names

# Membangun Model

In [ ]:
def build_feature_extraction(dropout_rate, learning_rate, optimizer_name, num_classes):

    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    base_model.trainable = False

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs)

    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Hyperparameter Tuning dengan Grid Search

In [ ]:
#Konfigurasi Grid Search
learning_rates = [1e-3, 1e-2]
optimizers = ["adam", "rmsprop"]
batch_sizes = [32, 64]
dropouts = [0.3, 0.4, 0.5]
epochs = 50  # max epoch, EarlyStopping aktif

results = []
best_val_acc = 0
best_model_path = "best_feature_extraction.h5"

training loop grid search

In [ ]:
experiment_id = 1

for lr in learning_rates:
    for opt in optimizers:
        for bs in batch_sizes:
            for dr in dropouts:

                print(f"\n=== Experiment {experiment_id} ===")
                print(f"LR={lr}, OPT={opt}, BS={bs}, DR={dr}")

                train_ds, val_ds, test_ds, class_names = load_dataset(bs)
                num_classes = len(class_names)

                model = build_feature_extraction(dr, lr, opt, num_classes)

                callbacks = [
                    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True),
                    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3)
                ]

                start_train = time.time()

                history = model.fit(
                    train_ds,
                    validation_data=val_ds,
                    epochs=epochs,
                    callbacks=callbacks,
                    verbose=1
                )

                training_time = time.time() - start_train

                val_loss, val_acc = model.evaluate(val_ds, verbose=0)

                # Simpan model terbaik
                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    model.save(best_model_path)

                results.append({
                    "Experiment": experiment_id,
                    "Learning Rate": lr,
                    "Optimizer": opt,
                    "Batch Size": bs,
                    "Dropout": dr,
                    "Val Accuracy": val_acc,
                    "Training Time (s)": training_time
                })

                experiment_id += 1


=== Experiment 1 ===
LR=0.001, OPT=adam, BS=32, DR=0.3
Found 11704 files belonging to 8 classes.
Found 617 files belonging to 8 classes.
Found 620 files belonging to 8 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 1093s 3s/step - accuracy: 0.8357 - loss: 0.4991 - val_accuracy: 0.9708 - val_loss: 0.0911 - learning_rate: 0.0010
Epoch 2/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 45s 122ms/step - accuracy: 0.9823 - loss: 0.0619 - val_accuracy: 0.9708 - val_loss: 0.0755 - learning_rate: 0.0010
Epoch 3/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 44s 121ms/step - accuracy: 0.9852 - loss: 0.0442 - val_accuracy: 0.9757 - val_loss: 0.0771 - learning_rate: 0.0010
Epoch 4/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 44s 120ms/step - accuracy: 0.9912 - loss: 0.0284 - val_accuracy: 0.9838 - val_loss: 0.0599 - learning_rate: 0.0010
Epoch 5/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 44s 120ms/step - accuracy: 0.9926 - loss: 0.0220 - val_accuracy: 0.9708 - val_loss: 0.0872 - learning_rate: 0.0010


=== Experiment 2 ===
LR=0.001, OPT=adam, BS=32, DR=0.4
Found 11704 files belonging to 8 classes.
Found 617 files belonging to 8 classes.
Found 620 files belonging to 8 classes.
Epoch 1/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 72s 162ms/step - accuracy: 0.8040 - loss: 0.5777 - val_accuracy: 0.9789 - val_loss: 0.0830 - learning_rate: 0.0010
Epoch 2/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 44s 121ms/step - accuracy: 0.9734 - loss: 0.0847 - val_accuracy: 0.9724 - val_loss: 0.0776 - learning_rate: 0.0010
Epoch 3/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 44s 119ms/step - accuracy: 0.9840 - loss: 0.0521 - val_accuracy: 0.9854 - val_loss: 0.0563 - learning_rate: 0.0010
Epoch 4/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 44s 121ms/step - accuracy: 0.9909 - loss: 0.0365 - val_accuracy: 0.9806 - val_loss: 0.0624 - learning_rate: 0.0010
Epoch 5/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 81s 119ms/step - accuracy: 0.9920 - loss: 0.0288 - val_accuracy: 0.9838 - val_loss: 0.0650 - learning_rate: 0.0010
Epoch 6/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 44s 119


=== Experiment 3 ===
LR=0.001, OPT=adam, BS=32, DR=0.5
Found 11704 files belonging to 8 classes.
Found 617 files belonging to 8 classes.
Found 620 files belonging to 8 classes.
Epoch 1/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 71s 159ms/step - accuracy: 0.7719 - loss: 0.6790 - val_accuracy: 0.9611 - val_loss: 0.1111 - learning_rate: 0.0010
Epoch 2/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 43s 119ms/step - accuracy: 0.9678 - loss: 0.1121 - val_accuracy: 0.9724 - val_loss: 0.0718 - learning_rate: 0.0010
Epoch 3/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 82s 118ms/step - accuracy: 0.9796 - loss: 0.0657 - val_accuracy: 0.9789 - val_loss: 0.0660 - learning_rate: 0.0010
Epoch 4/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 44s 121ms/step - accuracy: 0.9805 - loss: 0.0555 - val_accuracy: 0.9773 - val_loss: 0.0583 - learning_rate: 0.0010
Epoch 5/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 43s 118ms/step - accuracy: 0.9834 - loss: 0.0471 - val_accuracy: 0.9789 - val_loss: 0.0582 - learning_rate: 0.0010
Epoch 6/50
366/366 ━━━━━━━━━━━━━━━━━━━━ 43s 118


=== Experiment 18 ===
LR=0.005, OPT=adam, BS=64, DR=0.5
Found 11704 files belonging to 8 classes.
Found 617 files belonging to 8 classes.
Found 620 files belonging to 8 classes.
Epoch 1/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 67s 299ms/step - accuracy: 0.7604 - loss: 0.7597 - val_accuracy: 0.9417 - val_loss: 0.1593 - learning_rate: 0.0050
Epoch 2/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 40s 219ms/step - accuracy: 0.9456 - loss: 0.1632 - val_accuracy: 0.9757 - val_loss: 0.0792 - learning_rate: 0.0050
Epoch 3/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 40s 219ms/step - accuracy: 0.9640 - loss: 0.1091 - val_accuracy: 0.9708 - val_loss: 0.0889 - learning_rate: 0.0050
Epoch 4/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 40s 221ms/step - accuracy: 0.9612 - loss: 0.1128 - val_accuracy: 0.9741 - val_loss: 0.0837 - learning_rate: 0.0050
Epoch 5/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 40s 218ms/step - accuracy: 0.9640 - loss: 0.1020 - val_accuracy: 0.9708 - val_loss: 0.0852 - learning_rate: 0.0050
Epoch 6/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 40s 22

In [ ]:
import os

file_path = "/content/drive/MyDrive/Seminar Hasil/dataset_processed/train/yellow disease/Screenshot 2023-10-12 112056.png"
if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
else:
    print(f"The file '{file_path}' does NOT exist. Please verify the path and file name.")

Rekap Hasil Grid Search

In [ ]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="Val Accuracy", ascending=False)

results_df.head()

# Evaluasi Hasil

dilakukan pada test set

In [ ]:
best_model = tf.keras.models.load_model(best_model_path)

test_ds = load_dataset(batch_size=32)[2]  # load test
class_names = load_dataset(batch_size=32)[3]

test_loss, test_acc = best_model.evaluate(test_ds)
print("Test Accuracy:", test_acc)

In [ ]:
#confusion matrix
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = best_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

In [ ]:
#waktu inferensi
sample_batch = next(iter(test_ds))[0]

start_inf = time.time()
_ = best_model.predict(sample_batch)
inference_time = (time.time() - start_inf) / len(sample_batch)

print("Inference Time per Image (seconds):", inference_time)

In [ ]:
#ukuran file model
size_mb = os.path.getsize(best_model_path) / (1024*1024)
print("Model Size (MB):", size_mb)